In [6]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName('trip_count_by_zone_sql')
    .config("spark.ui.reverseProxyUrl", "4050")  # 이 부분이 추가된 부분입니다.
    .getOrCreate()
)

23/06/04 22:42:16 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [15]:
directory = "/Users/keemyohaan/Desktop/001.Python/004. Study/009.FastCampus/003.DE/01-spark/data"
trip_data = "fhvhv_tripdata_2020-03.csv"
zone_data = 'taxi+_zone_lookup.csv'

trip_data = spark.read.csv(f"file:///{directory}/{trip_data}", inferSchema = True, header= True)
zone_data = spark.read.csv(f"file:///{directory}/{zone_data}", inferSchema = True, header= True)

In [16]:
trip_data.show(5)

23/06/04 22:51:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , hvfhs_license_num, dispatching_base_num, pickup_datetime, PULocationID, DOLocationID, SR_Flag
 Schema: _c0, hvfhs_license_num, dispatching_base_num, pickup_datetime, PULocationID, DOLocationID, SR_Flag
Expected: _c0 but found: 
CSV file: file:///Users/keemyohaan/Desktop/001.Python/004.%20Study/009.FastCampus/003.DE/01-spark/data/fhvhv_tripdata_2020-03.csv
+---+-----------------+--------------------+-------------------+------------+------------+-------+
|_c0|hvfhs_license_num|dispatching_base_num|    pickup_datetime|PULocationID|DOLocationID|SR_Flag|
+---+-----------------+--------------------+-------------------+------------+------------+-------+
|  0|           HV0005|              B02510|2020-03-01 00:03:40|          81|         159|      N|
|  1|           HV0005|              B02510|2020-03-01 00:28:05|         168|         119|      N|
|  2|           HV0003|              B02764|2020-03-

In [17]:
zone_data.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [22]:
trip_data.createOrReplaceTempView('trip_data')
zone_data.createOrReplaceTempView('zone_data')

In [24]:
spark.sql('select * from trip_data limit 5').show()

23/06/04 22:55:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , hvfhs_license_num, dispatching_base_num, pickup_datetime, PULocationID, DOLocationID, SR_Flag
 Schema: _c0, hvfhs_license_num, dispatching_base_num, pickup_datetime, PULocationID, DOLocationID, SR_Flag
Expected: _c0 but found: 
CSV file: file:///Users/keemyohaan/Desktop/001.Python/004.%20Study/009.FastCampus/003.DE/01-spark/data/fhvhv_tripdata_2020-03.csv
+---+-----------------+--------------------+-------------------+------------+------------+-------+
|_c0|hvfhs_license_num|dispatching_base_num|    pickup_datetime|PULocationID|DOLocationID|SR_Flag|
+---+-----------------+--------------------+-------------------+------------+------------+-------+
|  0|           HV0005|              B02510|2020-03-01 00:03:40|          81|         159|      N|
|  1|           HV0005|              B02510|2020-03-01 00:28:05|         168|         119|      N|
|  2|           HV0003|              B02764|2020-03-

In [23]:
spark.sql('select * from zone_data limit 5').show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+



In [27]:
spark.sql('select zone_data.Borough, count(*) from trip_data join zone_data on trip_data.PULocationID = zone_data.LocationID group by 1').show()

+-------------+--------+
|      Borough|count(1)|
+-------------+--------+
|       Queens| 2437394|
|          EWR|     362|
|      Unknown|     845|
|     Brooklyn| 3735765|
|Staten Island|  178818|
|    Manhattan| 4953147|
|        Bronx| 2086597|
+-------------+--------+



맨하탄에서 가장 많이 승차가 일어나는군!

그럼 하차는 어떨까?

In [28]:
spark.sql(
    'select \
        zone_data.Borough, \
        count(*) \
    from trip_data join zone_data \
    on trip_data.DOLocationID = zone_data.LocationID group by 1').show()

+-------------+--------+
|      Borough|count(1)|
+-------------+--------+
|       Queens| 2468416|
|          EWR|   65066|
|      Unknown|  387760|
|     Brooklyn| 3696684|
|Staten Island|  177727|
|    Manhattan| 4553783|
|        Bronx| 2043492|
+-------------+--------+



공항에서 하차가 많이 일어나는군